In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
from tensorflow import keras
import numpy as np
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="feedback_db"
)
mycursor = mydb.cursor()
import tensorflow as tf

config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)
model = tf.keras.models.load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json', encoding='utf-8').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [2]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [3]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [4]:
from flask import Flask, render_template, request
import datetime

x = datetime.datetime.now()


app= Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def index():
    return render_template('test.html')

@app.route("/get",methods=['GET','POST'])
def process():
    user_input = request.args['msg']
    #if user_input !=(''):
    bot_response = chatbot_response(user_input)
    return str(bot_response)

@app.route("/feedback")
def feedback():
    return render_template('feedback.html')

if __name__=='__main__':
	app.run(debug=False,port=5002)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)


In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
from tkinter import ttk
import subprocess
from subprocess import call
import os
global x

x=0
      
def send():
        global x
        msg = EntryBox.get("1.0",'end-1c',).strip()
        EntryBox.delete("0.0",END)

        if msg == ('No') and x != 0 :
            messagebox.showinfo(title='Comment info', message='We hope that we are able to solve your problem .')
            base.destroy()
            x = 2
        elif msg == ('Yes') and x != 0:
            base.destroy()
            os.system('python feedback.py')
            x = 2
        elif msg != (''):
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "User: " + msg + '\n\n')
            ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
            res = chatbot_response(msg)
            ChatLog.insert(END, "ITBot: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)


def done():
        global x
        msg = EntryBox.get("1.0",'end-1c',).strip()
        EntryBox.delete("0.0",END)
        if x == 0:
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, 'ITBot: We hope we are able to solve your problem .' + '\n\n')
            ChatLog.insert(END, 'ITBot: We are conducting a survey base on your experience with our chatbot ?' + '\n\n')
            ChatLog.insert(END, 'ITBot: Type : Yes Or No ?' + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
            x=2




base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(bg="white", height="8", width="50", font="Verdana",foreground="#442265")
ChatLog.config(state=NORMAL)
ChatLog.insert(INSERT, 'ITBot: Welcome, How can i help you ?' + '\n\n')
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                        bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                        command= send )
DoneButton = Button(base, font=("Verdana",12,'bold'), text="Exit", width="12", height=5,
                        bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff' , command= done )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=45, width=120)
DoneButton.place(x=6, y=447, height=44, width=120)
base.mainloop()

In [ ]:
from flask import Flask, render_template, request
import datetime

x = datetime.datetime.now()


app= Flask(__name__)
app.static_folder = 'static'


@app.route("/")
def index():
    return render_template('index.html')

@app.route("/process",methods=['POST'])
def process():
    user_input=request.form['user_input']
    if user_input !=(''):
        bot_response = chatbot_response(user_input)
        print("ITBot: "+ bot_response)
        return render_template('index.html',user_input=user_input,bot_response=bot_response)

if __name__=='__main__':
	app.run(debug=False,port=5002)

In [1]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
